<a href="https://colab.research.google.com/github/huseynovnebi/Market-Basket-Analysis-Apriori/blob/main/Market_Basket_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from mlxtend.frequent_patterns import apriori, association_rules

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
df= pd.read_csv('/content/online_retail_II.csv')

In [6]:
df.shape

(1067371, 8)

In [7]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


In [9]:
df.isnull().sum()

,0
Invoice,0
StockCode,0
Description,4382
Quantity,0
InvoiceDate,0
Price,0
Customer ID,243007
Country,0


In [10]:
df[df['Customer ID'].isna()]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
263,489464,21733,85123a mixed,-96,2009-12-01 10:52:00,0.00,NaN,United Kingdom
283,489463,71477,short,-240,2009-12-01 10:52:00,0.00,NaN,United Kingdom
284,489467,85123A,21733 mixed,-192,2009-12-01 10:53:00,0.00,NaN,United Kingdom
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.00,NaN,United Kingdom
577,489525,85226C,BLUE PULL BACK RACING CAR,1,2009-12-01 11:49:00,0.55,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
1066997,581498,85099B,JUMBO BAG RED RETROSPOT,5,2011-12-09 10:26:00,4.13,NaN,United Kingdom
1066998,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,2011-12-09 10:26:00,4.13,NaN,United Kingdom
1066999,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,2011-12-09 10:26:00,4.96,NaN,United Kingdom
1067000,581498,85174,S/4 CACTI CANDLES,1,2011-12-09 10:26:00,10.79,NaN,United Kingdom


In [11]:
df = df.dropna(subset=['Description'])

In [12]:
df.isnull().sum()

,0
Invoice,0
StockCode,0
Description,0
Quantity,0
InvoiceDate,0
Price,0
Customer ID,238625
Country,0


In [13]:
num_columns = df.select_dtypes(['int','float']).drop(columns=['Customer ID'])
num_columns

,Quantity,Price
0,12,6.95
1,12,6.75
2,12,6.75
3,48,2.10
4,24,1.25
...,...,...
1067366,6,2.10
1067367,4,4.15
1067368,4,4.15
1067369,3,4.95


In [14]:
df[df['Quantity'] < 0]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.0,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.0,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321.0,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
...,...,...,...,...,...,...,...,...
1065910,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
1067002,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
1067176,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
1067177,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom


In [15]:
df = df[df['Quantity'] > 0]

In [16]:
df[df['Price'] < 0]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
179403,A506401,B,Adjust bad debt,1,2010-04-29 13:36:00,-53594.36,NaN,United Kingdom
276274,A516228,B,Adjust bad debt,1,2010-07-19 11:24:00,-44031.79,NaN,United Kingdom
403472,A528059,B,Adjust bad debt,1,2010-10-20 12:04:00,-38925.87,NaN,United Kingdom
825444,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
825445,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


In [17]:
df[df['Description'] == 'Adjust bad debt']

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
179403,A506401,B,Adjust bad debt,1,2010-04-29 13:36:00,-53594.36,NaN,United Kingdom
276274,A516228,B,Adjust bad debt,1,2010-07-19 11:24:00,-44031.79,NaN,United Kingdom
403472,A528059,B,Adjust bad debt,1,2010-10-20 12:04:00,-38925.87,NaN,United Kingdom
825443,A563185,B,Adjust bad debt,1,2011-08-12 14:50:00,11062.06,NaN,United Kingdom
825444,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
825445,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


In [18]:
df = df[~((df['Price'] < 0) & (df['Description'] == 'Adjust bad debt'))]
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [19]:
df[df['Quantity'] > 100]


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
217,489460,84598,BOYS ALPHABET IRON ON PATCHES,576,2009-12-01 10:46:00,0.21,16167.0,United Kingdom
223,489460,84568,GIRLS ALPHABET IRON ON PATCHES,288,2009-12-01 10:46:00,0.21,16167.0,United Kingdom
282,489465,84879,ASSORTED COLOUR BIRD ORNAMENT,160,2009-12-01 10:52:00,1.45,13767.0,United Kingdom
575,489523,84879,ASSORTED COLOUR BIRD ORNAMENT,800,2009-12-01 11:46:00,1.45,12931.0,United Kingdom
934,489539,22086,PAPER CHAIN KIT 50'S CHRISTMAS,160,2009-12-01 12:18:00,2.55,15061.0,United Kingdom
...,...,...,...,...,...,...,...,...
1065903,581487,21137,BLACK RECORD COVER FRAME,120,2011-12-09 09:44:00,3.39,15694.0,United Kingdom
1066747,581497,22355,CHARLOTTE BAG SUKI DESIGN,110,2011-12-09 10:23:00,2.46,NaN,United Kingdom
1067006,581501,22319,HAIRCLIPS FORTIES FABRIC ASSORTED,180,2011-12-09 10:46:00,0.19,12985.0,United Kingdom
1067163,581566,23404,HOME SWEET HOME BLACKBOARD,144,2011-12-09 11:50:00,3.26,18102.0,United Kingdom


In [20]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,1042723.0,11.019149,127.689626,1.0,1.00,3.0,10.00,80995.00
Price,1042723.0,4.072925,51.423182,0.0,1.25,2.1,4.13,25111.09
Customer ID,805620.0,15331.856250,1696.768395,12346.0,13982.00,15271.0,16805.00,18287.00


In [21]:
df['Quantity'].quantile(0.99)

100.0

In [22]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [23]:
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [24]:
def retail_data_prep(dataframe):
    dataframe.dropna(inplace=True)
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")
    return dataframe

In [25]:
df = retail_data_prep(df)

In [26]:
df.shape

(805620, 8)

In [27]:
df.isnull().sum()

,0
Invoice,0
StockCode,0
Description,0
Quantity,0
InvoiceDate,0
Price,0
Customer ID,0
Country,0


In [28]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,805620.0,11.842778,26.835632,1.0,2.00,5.00,12.00,318.50
Price,805620.0,2.949878,3.238458,0.0,1.25,1.95,3.75,36.94
Customer ID,805620.0,15331.856250,1696.768395,12346.0,13982.00,15271.00,16805.00,18287.00


In [29]:
df['Country'].value_counts()

,count
Country,
United Kingdom,725296
Germany,16703
EIRE,15745
France,13813
Netherlands,5093
Spain,3720
Belgium,3069
Switzerland,3012
Portugal,2446


The choice to filter by specific countries (like France in this case) is often based on the idea that markets vary significantly between countries, and focusing on one country can provide deeper insights or more relevant results for a particular analysis. Here's why someone might choose to focus on a specific country:

In [30]:
df_fr = df[df['Country'] == "France"]

In [31]:
df_fr.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
71,489439,22065,CHRISTMAS PUDDING TRINKET POT,12.0,2009-12-01 09:28:00,1.45,12682.0,France
72,489439,22138,BAKING SET 9 PIECE RETROSPOT,9.0,2009-12-01 09:28:00,4.95,12682.0,France
73,489439,22139,RETRO SPOT TEA SET CERAMIC 11 PC,9.0,2009-12-01 09:28:00,4.95,12682.0,France
74,489439,22352,LUNCHBOX WITH CUTLERY RETROSPOT,12.0,2009-12-01 09:28:00,2.55,12682.0,France
75,489439,85014A,BLACK/BLUE DOTS RUFFLED UMBRELLA,3.0,2009-12-01 09:28:00,5.95,12682.0,France


In [32]:
df_fr_gr=df_fr.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)

In [33]:
df_fr_gr.head(10)

Description,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,FLAMINGO LIGHTS,I LOVE LONDON MINI BACKPACK,LARGE SKULL WINDMILL,NINE DRAWER OFFICE TIDY,RED/WHITE DOT MINI CASES,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TRELLIS COAT RACK,...,YELLOW GIANT GARDEN THERMOMETER,YELLOW RED FLOWER PIGGY BANK,YOU'RE CONFUSING ME METAL SIGN,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
Invoice,,,,,,,,,,,,,,,,,,,,,
489439,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489557,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489883,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
490139,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
490152,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
490458,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
490684,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
490959,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
491698,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
frequent_itemsets = apriori(df_fr_gr,
                            min_support=0.01,
                            use_colnames=True)

In [35]:
frequent_itemsets.sort_values("support", ascending=False).head(20)

,support,itemsets
305,0.758958,(POSTAGE)
353,0.210098,(RED TOADSTOOL LED NIGHT LIGHT)
2411,0.187296,"(RED TOADSTOOL LED NIGHT LIGHT, POSTAGE)"
380,0.175896,(ROUND SNACK BOXES SET OF4 WOODLAND )
293,0.169381,(PLASTERS IN TIN CIRCUS PARADE )
2428,0.159609,"(ROUND SNACK BOXES SET OF4 WOODLAND , POSTAGE)"
298,0.157980,(PLASTERS IN TIN WOODLAND ANIMALS)
213,0.153094,(LUNCH BOX WITH CUTLERY RETROSPOT )
2199,0.138436,"(PLASTERS IN TIN CIRCUS PARADE , POSTAGE)"
444,0.138436,(SET/6 RED SPOTTY PAPER CUPS)


In [36]:
rules = association_rules(frequent_itemsets,
                          metric="support",
                          min_threshold=0.01)

In [37]:
rules[(rules["support"]>0.05) & (rules["confidence"]>0.1) & (rules["lift"]>5)]. \
sort_values("confidence", ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
22382,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...",(SET/6 RED SPOTTY PAPER CUPS),0.073290,0.138436,0.071661,0.977778,7.063007,1.0,0.061515,38.770358,0.926306,0.511628,0.974207,0.747712
22384,"(SET/20 RED RETROSPOT PAPER NAPKINS , SET/6 RE...",(SET/6 RED SPOTTY PAPER PLATES),0.073290,0.127036,0.071661,0.977778,7.696866,1.0,0.062351,39.283388,0.938888,0.556962,0.974544,0.770940
46219,"(SET/20 RED RETROSPOT PAPER NAPKINS , SET/6 RE...",(SET/6 RED SPOTTY PAPER PLATES),0.060261,0.127036,0.058632,0.972973,7.659044,1.0,0.050977,32.299674,0.925188,0.455696,0.969040,0.717256
46217,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...",(SET/6 RED SPOTTY PAPER CUPS),0.060261,0.138436,0.058632,0.972973,7.028299,1.0,0.050290,31.877850,0.912719,0.418605,0.968630,0.698251
4594,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.127036,0.138436,0.115635,0.910256,6.575264,1.0,0.098049,9.600279,0.971305,0.771739,0.895836,0.872775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21630,(SET/6 RED SPOTTY PAPER CUPS),"(SET/20 RED RETROSPOT PAPER NAPKINS , POSTAGE)",0.138436,0.079805,0.060261,0.435294,5.454502,1.0,0.049213,1.629513,0.947887,0.381443,0.386320,0.595198
11287,(SPACEBOY LUNCH BOX ),"(DOLLY GIRL LUNCH BOX, POSTAGE)",0.130293,0.073290,0.055375,0.425000,5.798889,1.0,0.045825,1.611670,0.951531,0.373626,0.379525,0.590278
46228,(SET/6 RED SPOTTY PAPER CUPS),"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...",0.138436,0.060261,0.058632,0.423529,7.028299,1.0,0.050290,1.630160,0.995537,0.418605,0.386563,0.698251
10309,(SPACEBOY LUNCH BOX ),"(CIRCUS PARADE LUNCH BOX , POSTAGE)",0.130293,0.071661,0.050489,0.387500,5.407386,1.0,0.041152,1.515655,0.937175,0.333333,0.340219,0.546023


In [54]:
strong_rules = rules[(rules["confidence"] > 0.8) &
                     (rules["lift"] > 1.5) &
                     (rules["support"] > 0.05)]
strong_rules.sort_values("confidence", ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
22382,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...",(SET/6 RED SPOTTY PAPER CUPS),0.073290,0.138436,0.071661,0.977778,7.063007,1.0,0.061515,38.770358,0.926306,0.511628,0.974207,0.747712
22384,"(SET/20 RED RETROSPOT PAPER NAPKINS , SET/6 RE...",(SET/6 RED SPOTTY PAPER PLATES),0.073290,0.127036,0.071661,0.977778,7.696866,1.0,0.062351,39.283388,0.938888,0.556962,0.974544,0.770940
46217,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...",(SET/6 RED SPOTTY PAPER CUPS),0.060261,0.138436,0.058632,0.972973,7.028299,1.0,0.050290,31.877850,0.912719,0.418605,0.968630,0.698251
46219,"(SET/20 RED RETROSPOT PAPER NAPKINS , SET/6 RE...",(SET/6 RED SPOTTY PAPER PLATES),0.060261,0.127036,0.058632,0.972973,7.659044,1.0,0.050977,32.299674,0.925188,0.455696,0.969040,0.717256
4594,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.127036,0.138436,0.115635,0.910256,6.575264,1.0,0.098049,9.600279,0.971305,0.771739,0.895836,0.872775
21663,"(SET/6 RED SPOTTY PAPER PLATES, POSTAGE)",(SET/6 RED SPOTTY PAPER CUPS),0.107492,0.138436,0.096091,0.893939,6.457398,1.0,0.081210,8.123313,0.946926,0.641304,0.876898,0.794029
5354,"(ALARM CLOCK BAKELIKE GREEN, POSTAGE)",(ALARM CLOCK BAKELIKE RED ),0.058632,0.068404,0.050489,0.861111,12.588624,1.0,0.046478,6.707492,0.977899,0.659574,0.850913,0.799603
4595,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.138436,0.127036,0.115635,0.835294,6.575264,1.0,0.098049,5.300140,0.984158,0.771739,0.811326,0.872775
156,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED ),0.068404,0.068404,0.057003,0.833333,12.182540,1.0,0.052324,5.589577,0.985315,0.714286,0.821096,0.833333
157,(ALARM CLOCK BAKELIKE RED ),(ALARM CLOCK BAKELIKE GREEN),0.068404,0.068404,0.057003,0.833333,12.182540,1.0,0.052324,5.589577,0.985315,0.714286,0.821096,0.833333


In [43]:
frequent_itemsets.iloc[2411]

,2411
support,0.187296
itemsets,"(RED TOADSTOOL LED NIGHT LIGHT, POSTAGE)"


In [52]:
specific_rules = rules[rules['antecedents'].apply(lambda x: 'RED TOADSTOOL LED NIGHT LIGHT' in x) &
                       rules['consequents'].apply(lambda x: 'POSTAGE' in x)]
specific_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
3792,(RED TOADSTOOL LED NIGHT LIGHT),(POSTAGE),0.210098,0.758958,0.187296,0.891473,1.174602,1.0,0.027841,2.221033,0.188185,0.239583,0.549759,0.569127
4815,"(RED TOADSTOOL LED NIGHT LIGHT, 4 TRADITIONAL ...",(POSTAGE),0.016287,0.758958,0.014658,0.900000,1.185837,1.0,0.002297,2.410423,0.159308,0.019272,0.585135,0.459657
4817,(RED TOADSTOOL LED NIGHT LIGHT),"(POSTAGE, 4 TRADITIONAL SPINNING TOPS)",0.210098,0.053746,0.014658,0.069767,1.298097,1.0,0.003366,1.017223,0.290722,0.058824,0.016932,0.171247
5702,"(RED TOADSTOOL LED NIGHT LIGHT, ALARM CLOCK BA...",(POSTAGE),0.014658,0.758958,0.013029,0.888889,1.171197,1.0,0.001905,2.169381,0.148347,0.017131,0.539039,0.453028
5705,(RED TOADSTOOL LED NIGHT LIGHT),"(ALARM CLOCK BAKELIKE GREEN, POSTAGE)",0.210098,0.058632,0.013029,0.062016,1.057709,1.0,0.000711,1.003607,0.069072,0.050955,0.003594,0.142119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68538,"(RED TOADSTOOL LED NIGHT LIGHT, RED RETROSPOT ...","(STRAWBERRY LUNCH BOX WITH CUTLERY, POSTAGE, P...",0.034202,0.027687,0.011401,0.333333,12.039216,1.0,0.010454,1.458469,0.949410,0.225806,0.314350,0.372549
68542,"(RED TOADSTOOL LED NIGHT LIGHT, PLASTERS IN TI...","(STRAWBERRY LUNCH BOX WITH CUTLERY, RED RETROS...",0.052117,0.017915,0.011401,0.218750,12.210227,1.0,0.010467,1.257068,0.968581,0.194444,0.204498,0.427557
68546,"(RED TOADSTOOL LED NIGHT LIGHT, STRAWBERRY LUN...","(RED RETROSPOT MINI CASES, POSTAGE, PLASTERS I...",0.043974,0.022801,0.011401,0.259259,11.370370,1.0,0.010398,1.319218,0.954003,0.205882,0.241975,0.379630
68547,"(RED TOADSTOOL LED NIGHT LIGHT, PLASTERS IN TI...","(STRAWBERRY LUNCH BOX WITH CUTLERY, RED RETROS...",0.057003,0.016287,0.011401,0.200000,12.280000,1.0,0.010472,1.229642,0.974093,0.184211,0.186755,0.450000
